# Квантовый оптимальный контроль для эффективного Гамильтониана

Daniel Rosseau Qianqian Ha and Tim Byrnes, Phys.Rev.A 90, 052315 (2014)

$$ 
H 
    = \frac{ H_\mathrm{ eff } }{ \Omega } 
    = \frac{ \omega }{ \Omega } \underbrace{ \left( S^z_1 +  S^z_2 \right) }_{ H_\mathrm{ control } }
    - \underbrace{ \left( 
        S^z_1 S^z_2 
        + \frac{ \left( S^z_1 \right)^2 }{ 2 }
        + \frac{ \left( S^z_2 \right)^2 }{ 2 }
    \right) }_{ H_\mathrm{ drift } }
$$



In [1]:
import math

import matplotlib.pyplot as plt
import numpy as np
import qutip
from qutip.control.pulseoptim import create_pulse_optimizer, optimize_pulse

In [2]:
import pathlib, sys

sys.path.append(str(pathlib.Path(sys.path[0]).parent / "libs"))

In [3]:
%reload_ext autoreload
%autoreload 2

import bec
from tools.jupyter import print_model_info

In [4]:
N_BOSONS = 3
PHASE = 0  # np.pi / 4
model_default = bec.BEC_Qubits.init_default(n_bosons=N_BOSONS, phase=PHASE)


def omega(m):
    return m.g**2 / 2 / m.delta + m.Omega * m.n_bosons / 2


print_model_info(model_default)
print("omega =", omega(model_default))
print("omega / Omega =", omega(model_default) / model_default.Omega)

BEC_Qubits(n_bosons=3, coupling_strength=1, transition_ampl=1, transition_freq=11, resonance_freq=1, phase=0, excitation_level=False)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

omega = 0.05075
omega / Omega = 101.5


In [5]:
def h_drift(m):
    return (
        bec.sz(m, n=2, k=0) * bec.sz(m, n=2, k=1)
        + bec.sz(m, n=2, k=0) ** 2 / 2
        + bec.sz(m, n=2, k=1) ** 2 / 2
    )

[Quantum Optimal Control with Qutip](https://qutip.org/docs/latest/guide/guide-control.html)

$$ H(t) = H_0 + \alpha(t) H_1 + \beta(t) H_2 $$

In [6]:
model = model_default

H0 = h_drift(model)
H1 = bec.sz(model, n=2, k=0)
H2 = bec.sz(model, n=2, k=1)

In [7]:
psi_initial = (
    bec.coherent_state_constructor(model, n=2, k=0)
    * bec.coherent_state_constructor(model, 2, 1, math.sqrt(1 / 10), math.sqrt(9 / 10))
    * bec.vacuum_state(model, n=2)
)

In [8]:
def states_under_hzz_teor(t):
    """
    Эволюционные состояния (теория) системы двух кубитов, с начальным суперпозиционным состоянием.
    """

    def alpha(k, t):
        return np.exp(1j * (model.n_bosons - 2 * k) * model.Omega * t) / math.sqrt(2)

    def beta(k, t):
        return alpha(k, t).conjugate()

    return sum(
        (
            math.sqrt(math.comb(model.n_bosons, k))
            * bec.coherent_state_constructor(model, 2, 0, alpha(k, t), beta(k, t))
            * bec.fock_state_constructor(model, 2, 1, k)
            * bec.vacuum_state(model)
        )
        for k in range(model.n_bosons + 1)
    ) / math.sqrt(2**model.n_bosons)

In [9]:
psi_target = states_under_hzz_teor(2 / model.Omega)

In [10]:
qutip.entropy_vn(qutip.ptrace(psi_target, [0, 1]))

1.1420491198826328

In [11]:
nt = 100
t_total = 2 / model.Omega

In [12]:
%%time
# Fidelity error target
fid_err_targ = 1e-5
# Maximum iterations for the optisation algorithm
max_iter = 500
# Maximum (elapsed) time allowed in seconds
max_wall_time = 30
# Minimum gradient (sum of gradients squared)
# as this tends to 0 -> local minima has been found
min_grad = 1e-10
# pulse type alternatives: RND|ZERO|LIN|SINE|SQUARE|SAW|TRIANGLE|
init_pulse_type = "RND"

result = optimize_pulse(
    H0,
    [H1, H2],
    psi_initial,
    psi_target,
    num_tslots=nt,
    evo_time=t_total,
    fid_err_targ=fid_err_targ,
    max_iter=max_iter,
    max_wall_time=max_wall_time,
    min_grad=min_grad,
    init_pulse_type=init_pulse_type,
    # dyn_type='SYMPL',
    gen_stats=True,
)

/Users/kephircheek/venvs/science/emqwuqoc-dev/lib/python3.11/site-packages/scipy/linalg/_expm_frechet.py:277: RuntimeWarning: overflow encountered in add
  L = np.dot(R, L) + np.dot(L, R)


ValueError: array must not contain infs or NaNs

In [ ]:
result.stats.report()

In [ ]:
result.fidelity